# Modeling Entrainment of a WB Neuron (with BMTK)

In this notebook, we model 1:1 entrainment of a standard WB neuron under a constant frequency of excitatory input pulses. The entrainment is 1:1 because the WB neuron will fire at the same frequency as the input.


## Cell Definition
A standard WB cell is used in this example. 

**WB Cell**
* $\alpha_{n} = -0.01*(v+34)/(e^{-0.1*(v+34)}-1)$
* $\beta_{n} = 0.125*e^{-(v+44)/80}$
* $\alpha_{m} = 0.1*(v+35)/(1-e^{-(v+35)/10})$
* $\beta_{m} = 4*e^{-(v+60)/18}$
* $\alpha_{h} = 0.07*e^{-(v+58)/20}$
* $\beta_{h} = 1/(e^{-0.1*(v+28)}+1)$

### 1.Building the Network
There is only 1 cell in this network, thus only a WB cell will be added. No network connections are necessary

In [1]:
import numpy as np
import logging

from bmtk.builder.networks import NetworkBuilder

net = NetworkBuilder('network')
net.add_nodes(cell_name='WB',     
        model_type='biophysical',
        model_template='hoc:WBInhCell',  # WBInhCell hoc definition
        morphology='blank.swc')

c:\users\strou\anaconda3\envs\py2env\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Building
After defining our network, we build and save it in a network folder.

In [2]:
net.build()
net.save_nodes(output_dir='network')

#### External Input
We then create a 'virtual' cell that supplies the excitatory synaptic input to our WB cell.

In [3]:
extnet = NetworkBuilder('extnet')
extnet.add_nodes(N=1,pop_name='Inp',
            potential='exc',
            model_type='virtual')

Add an excitatory connection from our virtual cell to the WB cell using tau1=0.5 ms and t2=2 ms. These parameters are specified in ExcToInh.json

In [4]:
extnet.add_edges(source={'pop_name':'Inp'}, target=net.nodes(),
                   connection_rule=1,
                   syn_weight=0.195,               #synaptic weight specified in Borger's book ch 23
                   delay=0,
                   weight_function=None,
                   target_sections=['basal', 'apical'],
                   distance_range=[0.0, 150.0],
                   dynamics_params='ExcToInh.json',
                   model_template='exp2syn')

In [5]:
extnet.build()
extnet.save_nodes(output_dir='network')
extnet.save_edges(output_dir='network')

#### Adding Spike Train

In [6]:
from bmtk.utils.reports.spike_trains import PoissonSpikeGenerator

psg = PoissonSpikeGenerator(population='extnet')
psg.add(node_ids=range(1),  # Have 1 node to match mthalamus
        firing_rate=20.0,    # T=50 ms, or 20 Hz
        times=(0.0, 0.8))    # Firing starts at 0 s up to 3 s
psg.to_sonata('input/PoissonSpike.h5')

In [7]:
from bmtk.utils.reports.spike_trains import SpikeTrains

spikes = SpikeTrains(population='extnet')
spikes.add_spikes(node_ids=0, timestamps=[50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800])


In [7]:
print('Number of spikes: {}'.format(psg.n_spikes()))
print('Units: {}'.format(psg.units()))

psg.to_dataframe().head()


Number of spikes: 8
Units: ms


,node_ids,timestamps,population
0,0,43.881578,extnet
1,0,238.799072,extnet
2,0,346.862505,extnet
3,0,444.280148,extnet
4,0,466.297602,extnet


In [11]:
from bmtk.utils.sim_setup import build_env_bionet

build_env_bionet(base_dir='.',
                 network_dir='network',
                 tstop=800.0, dt=0.1,
                 report_vars=['v'],     # Record membrane potential and calcium (default soma)
                 spikes_inputs=[('extnet')],   # Name of population which spikes will be generated for
                                #'input/PoissonSpike.h5')],
                 include_examples=False,    # Copies components files
                 compile_mechanisms=False   # Will try to compile NEURON mechanisms
                )

### Run the Simulation

In [13]:
from bmtk.simulator import bionet


conf = bionet.Config.from_json('simulation_config.json')
conf.build_env()
net = bionet.BioNetwork.from_config(conf)
sim = bionet.BioSimulator.from_config(conf, network=net)
sim.run()

WindowsError: [Error 32] The process cannot access the file because it is being used by another process: u'C:\\Users\\strou\\Documents\\MIZNEURO\\rhythms-master\\BMTK Examples\\23-WBEntrainment\\output\\log.txt'

In [ ]:
from bmtk.analyzer.spike_trains import to_dataframe

results_df = to_dataframe(config_file='simulation_config.json')
print('Number of Spikes: {}'.format(len(results_df)))
results_df.head()

In [ ]:
from bmtk.analyzer.compartment import plot_traces

_ = plot_traces(config_file='simulation_config.json', node_ids=[0], report_name='v_report')